The purpose of this notebook is to fetch roll call votes ("afstemninger") from the Danish Parliament's (Folketinget) public website (ft.dk). The website has a public API (OData, oda.ft.dk) which can be used to get the votes. The main challenge is that the data model seems to mirror their internaæ case management system and hence some wrangling and joining is necessary before arriving at an analytics friendly dataset.  

In [1]:
# Hent alle vedtagne lovforslag - fremsat af regeringen - i et givent sæt af samlinger

import requests
import pandas as pd
import numpy as np

# typeid filtrerer for sagstype, hvor 3 er lovforslag
# statusid filtrerer for sagsstatus, hvor 11 er stadfæstet
# kategoriid filtrerer for sagskategori, hvor 13 er regeringsforslag
# periode filtrer for samling, hvor 155 fx. er samling 20211 med start 2021-10-05 og slut 2021-10-04

perioder = [153, 155]
df = pd.DataFrame()
for periode in perioder:
    url = 'https://oda.ft.dk/api/Sag?$filter=typeid eq 3 and statusid eq 11 and kategoriid eq 13 and periodeid eq ' + str(periode) + '&$format=json'
    sag = requests.get(url)
    data = sag.json()['value']
    if len(df.columns) == 0:
        df = pd.DataFrame(data)
    else:
        df = pd.concat([df,pd.DataFrame(data)])
    while True:
        if "odata.nextLink" not in sag.json():
            break
        next_url = sag.json()['odata.nextLink']
        sag = requests.get(next_url)
        data = sag.json()['value']
        df = pd.concat([df,pd.DataFrame(data)])

In [2]:
# Hent ministerområde tilknyttet hvert lovforslag. Ministerområdet kan ikke findes direkte på sagen men skal hentes som aktør-relation
# For hvert enkelt lovforslags kaldes api'et og ministerområdet tilføjes til datasættet

def query_area(caseid):
    #rolleid filtrerer for rolle, hvor 6 er ministerområde
    url = 'https://oda.ft.dk/api/SagAktør?$select=aktørid&$filter=rolleid eq 6 and sagid eq ' + str(caseid) + '&$format=json'
    sag = requests.get(url)
    data = sag.json()['value']
    return pd.DataFrame(data)["aktørid"][0]

df['ministeromraade_id'] = df.apply(lambda x: query_area(x['id']), axis=1)

In [3]:
# Hent liste over ministerområdet for at kunne konvertere id til navn
# typeid filtrerer for sagsaktør, hvor 1 er ministerområde
url = 'https://oda.ft.dk/api/Aktør?$filter=typeid eq 1&$format=json'
sag = requests.get(url)
data = sag.json()['value']
departments = pd.DataFrame(data)
departments = departments[['id','navn']]
departments = departments.rename(columns={'id': 'ministeromraade_id', 'navn': 'ministeromraade'})

In [4]:
# Flet ministerområde ind på datasæt via aktørid
df = pd.merge(df,departments,how='left',on='ministeromraade_id')

In [9]:
# Afgrænse data til kolonner relevante for det videre arbejde med at opsætte partitesten
df2 = df[["id", "periodeid", "nummer", "titel", "titelkort", "resume", "afstemningskonklusion","ministeromraade"]].copy()

In [10]:
# Hvert lovforslag har en afstemningskonklusion. Denne celle omdanner denne opsummering til enkeltkolonner med hvert partis stemme.
# Partierne og de bogstaver, som de identificeres ved i ft data kan findes på ft.dk
# https://www.ft.dk/da/partier/om-politiske-partier/partigruppernes-bogstaver

# Denne funktion læser en opsummering af afstemningsresultatet og udtrækker herfra hvad et parti stemte (for, imod, hverken/eller)
# Den understøtter ikke, hvis et parti registrerer som havende afgivet f.x. en for stemme "ved en fejl"
def extract_party_vote(roll_call, party):
    if roll_call == None:
        return np.nan
    votes_for = ''
    votes_against = ''
    votes_neither = ''
    if roll_call.find('For stemte 0') == -1:
        start = roll_call.find('For stemte')+10
        start = roll_call.find('(', start)+1
        end = roll_call.find(', imod')-1
        votes_for = roll_call[start:end]
        votes_for = votes_for.replace(' og ', ', ').replace(' ','')
        #print('F:', votes_for)
    if roll_call.find(', imod stemte 0') == -1:
        start = roll_call.find('imod stemte')+11
        start = roll_call.find('(', start)+1
        end = roll_call.find(', hverken for eller imod stemte')-1
        votes_against = roll_call[start:end]
        votes_against = votes_against.replace(' og ', ', ').replace(' ','')
        #print('A:', votes_against)
    if roll_call.find('hverken for eller imod stemte 0') == -1:
        start = roll_call.find('hverken for eller imod stemte')+11
        start = roll_call.find('(', start)+1
        end = roll_call.find(').', start)-1
        votes_neither = roll_call[start:end]
        votes_neither = votes_neither.replace(' og ', ', ').replace(' ','')
        #print('N: ', votes_neither)
    if party in votes_against.split(','):
        return 0
    if party in votes_for.split(','):
        return 1
    if party in votes_neither.split(','):
        return 2
    return np.nan

parties = ['S','V','SF','RV','EL','DD','KF','DF','NB','LA','FG','ALT','KD','M']
for party in parties:
    df2['stem_' + party] = df2.apply(lambda x: extract_party_vote(x['afstemningskonklusion'], party), axis=1)

In [11]:
# Nulstil index og gem data i parquet format
# Parquet formattet er mere robust end csv ift. at holde datatyper under kontrol mv.
df2['raekkenr'] = df2.reset_index().index
df2.to_parquet('data.parquet')

In [8]:
# Denne celle indeholder forskellige afstemningskonklusioner, som blev brugt til at teste om funktionen er korrekt
test_roll_call_votes = [
    'Forslaget blev vedtaget. For stemte 69 (S, DF, SF, RV, EL, LA, FG og ALT), imod stemte 33 (V, KF og NB), hverken for eller imod stemte 0.',
    'Forslaget blev vedtaget. For stemte 70 (S, V, DF, KF, NB og LA), imod stemte 14 (RV, EL og FG), hverken for eller imod stemte 12 (SF og 3 EL (ved en fejl)).',
    'Forslaget blev vedtaget. For stemte 72 (S, V, KF, DF, NB, Karina Adsbøl (UFG), Marie Krarup (UFG) og Orla Østerby (UFG)), imod stemte 31 (SF, RV, EL, LA, FG og ALT), hverken for eller imod stemte 0.',
    'Forslaget blev vedtaget. For stemte 66 (S, SF, RV, EL, KF (ved en fejl), LA, FG og ALT), imod stemte 11 (DF og NB), hverken for eller imod stemte 25 (V, KD).',
    'Forslaget blev vedtaget. For stemte 99 (S, V, DF, SF, RV, EL, KF, NB, LA, FG og ALT), imod stemte 0, hverken for eller imod stemte 0.',
    'Forslaget blev vedtaget. For stemte 100 (S, V, DF, SF, RV, EL, KF, NB, LA, FG og ALT), imod stemte 0, hverken for eller imod stemte 0.Ved en fejl er Rasmus Helveg Petersens (RV) stemme registreret i Henrik Dam Kristensens (S) navn.'
]
#print(test_roll_call_votes[2], ':', extract_party_vote(test_roll_call_votes[2], 'S'))
#for x in test_roll_call_votes:
#    print(extract_party_vote(x, 'ALT'), ':', x)
